In [57]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.3
H2O_cluster_version_age:,"14 days, 15 hours and 15 minutes"
H2O_cluster_name:,bonay
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,9 Gb
H2O_cluster_total_cores:,11
H2O_cluster_allowed_cores:,11
H2O_cluster_status:,"locked, healthy"


In [40]:
from h2o.automl import H2OAutoML

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

In [38]:

# Cargar el dataset
file_path = 'dataset/itesal.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
label = 'CATEGORIA_FACTURA'
train = h2o.H2OFrame(train_data)
test = h2o.H2OFrame(test_data)
y = "CATEGORIA_FACTURA"
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()    
x = list(train.columns)
x.remove(y)

aml1 = H2OAutoML(max_models=20, seed=1, max_runtime_secs=1200, sort_metric='mean_per_class_error', distribution='multinomial')
aml1.train(x=x, y=y, training_frame=train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
20:52:05.693: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_1_20240625_205205_model_5


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    40                 200                         1.57917e+07            15           17           16.985        88            2166          1548.43

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.001755976362125208
RMSE: 0.041904371635012115
LogLoss: 0.019311063163454125
Mean Per-Class Error: 0.0020230238672197627
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1    0    1      2      Error        Rate
----  ----  ---  -----  -----  -----------  -----------
140   0     0    1      0      0.0070922    1 / 141
0     5533  0    0      1      0.000180701  1 / 5,534
0     0     593  1      0      0.0016835    1 / 594
0     0     0    18505  9      0.000486119  9 / 18,514
0     0     0    26     38630  0.000672599  26 / 38,656
140   5533  593  18533  38640  0.000599001  38 / 63,439

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.999401
2    1
3    1
4    1
5    1

ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.09471418168893858
RMSE: 0.3077566923544289
LogLoss: 0.3595004039921246
Mean Per-Class Error: 0.37181989908229546
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1    0    1      2      Error      Rate
----  ----  ---  -----  -----  ---------  --------------
3     0     0    50     88     0.978723   138 / 141
0     5392  32   23     87     0.0256596  142 / 5,534
0     29    261  204    100    0.560606   333 / 594
6     2     21   14282  4203   0.228584   4,232 / 18,514
7     10    6    2510   36123  0.0655267  2,533 / 38,656
16    5433  320  17069  40601  0.116301   7,378 / 63,439

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.883699
2    0.990842
3    0.997084
4    0.998613
5    1

Cross-Validation Metrics Summary: 
                         mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.883447  0.00218956  0.88627       0.882409      0.883354      0.880517      0.884685
aic                      nan       0           nan           nan           nan           nan           nan
auc                      nan       0           nan           nan           nan           nan           nan
err                      0.116553  0.00218956  0.11373       0.117591      0.116646      0.119483      0.115315
err_count                1478.8    27.7975     1443          1492          1480          1516          1463
loglikelihood            nan       0           nan           nan           nan           nan           nan
logloss                  0.376824  0.01064     0.359837      0.382659      0.373373      0.381702      0.386548
max_per_class_error      0.985961  0.0192292   1             0.964286      0.965517

In [42]:
aml1.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GBM_grid_1_AutoML_1_20240625_205205_model_5,0.37182,0.3595,0.307757,0.0947142
GBM_4_AutoML_1_20240625_205205,0.37355,0.314739,0.296861,0.0881265
GBM_2_AutoML_1_20240625_205205,0.373924,0.318077,0.301319,0.0907931
GBM_3_AutoML_1_20240625_205205,0.373973,0.31398,0.299194,0.0895172
GBM_5_AutoML_1_20240625_205205,0.377381,0.325664,0.304844,0.0929301
GBM_1_AutoML_1_20240625_205205,0.378317,0.322348,0.301318,0.0907924
DRF_1_AutoML_1_20240625_205205,0.380491,0.398027,0.299834,0.0899003
GBM_grid_1_AutoML_1_20240625_205205_model_2,0.381609,0.348274,0.319888,0.102328
GBM_grid_1_AutoML_1_20240625_205205_model_3,0.384377,0.335493,0.310835,0.0966186
GBM_grid_1_AutoML_1_20240625_205205_model_4,0.39716,0.363375,0.318526,0.101459


In [43]:
#We need to get the predictions from the model: accuracy, precision, recall, f1-score
predictions = aml1.predict(test)
predictions = predictions.as_data_frame()
predictions = predictions['predict'].values
test = test.as_data_frame()
test = test['CATEGORIA_FACTURA'].values
#show the results
print("Accuracy: ", accuracy_score(test, predictions))
print("Precision: ", precision_score(test, predictions, average='weighted'))
print("Recall: ", recall_score(test, predictions, average='weighted'))
print("F1-Score: ", f1_score(test, predictions, average='weighted'))
print(classification_report(test, predictions))
print(confusion_matrix(test, predictions))

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Accuracy:  0.8936948297604035
Precision:  0.8914209361082124
Recall:  0.8936948297604035
F1-Score:  0.8912924578120646
              precision    recall  f1-score   support

          -2       0.38      0.06      0.10        50
          -1       1.00      0.97      0.98      1365
           0       0.81      0.49      0.61       140
           1       0.85      0.80      0.83      4702
           2       0.90      0.94      0.92      9603

    accuracy                           0.89     15860
   macro avg       0.79      0.65      0.69     15860
weighted avg       0.89      0.89      0.89     15860

[[   3    2    1   16   28]
 [   0 1320    9    6   30]
 [   0    2   68   42   28]
 [   1    0    5 3764  932]
 [   4    0    1  579 9019]]


/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'NUMERO_SERIE' has levels not trained on: ["R73"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECHA_FACTURA' has levels not trained on: ["02/07/2022", "06/04/2023", "13/04/2022", "13/08/2019", "14/06/2018", "17/04/2018", "23/01/2018", "24/04/2020", "26/08/2021", "29/06/2023", "29/12/2020", "30/08/2022"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FORMA_COBRO' has levels not trained on: ["CF63", "T60D"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'LIQUIDO_FACTURA' has levels not trained on: ["0,21", "0,4", "0,52", "0,56"

In [44]:
# Cargar el dataset
file_path = 'dataset/safta.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
label = 'CATEGORIA_FACTURA'

train = h2o.H2OFrame(train_data)
test = h2o.H2OFrame(test_data)
y = "CATEGORIA_FACTURA"
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()    
x = list(train.columns)
x.remove(y)

aml2 = H2OAutoML(max_models=20, seed=1, max_runtime_secs=1200, sort_metric='mean_per_class_error', distribution='multinomial')
aml2.train(x=x, y=y, training_frame=train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
21:12:10.146: AutoML: XGBoost is not available; skipping it.
21:12:10.148: _train param, Dropping bad and constant columns: [CENTRO_CONTABLE]

███████████
21:15:17.545: _train param, Dropping bad and constant columns: [CENTRO_CONTABLE]

██
21:15:55.501: _train param, Dropping bad and constant columns: [CENTRO_CONTABLE]

██
21:16:28.779: _train param, Dropping bad and constant columns: [CENTRO_CONTABLE]

█
21:16:48.221: _train param, Dropping bad and constant columns: [CENTRO_CONTABLE]

█
21:17:09.267: _train param, Dropping bad and constant columns: [CENTRO_CONTABLE]

█
21:17:36.850: _train param, Dropping bad and constant columns: [CENTRO_CONTABLE]

███
21:18:31.836: _train param, Dropping bad and constant columns: [CENTRO_CONTABLE]

█
21:18:50.344: _train param, Dropping bad and

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_2_20240625_211210_model_5


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    45                 225                         8.76254e+06            8            17           16.9422       21            1515          941.791

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.00046689267624429196
RMSE: 0.021607699466724632
LogLoss: 0.0054089649085577
Mean Per-Class Error: 0.002099738755280091
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1     0    1     2      Error        Rate
----  -----  ---  ----  -----  -----------  -----------
39    0      0    0     0      0            0 / 39
0     71755  0    0     0      0            0 / 71,755
0     1      795  7     0      0.00996264   8 / 803
0     0      0    8431  4      0.000474215  4 / 8,435
0     0      0    1     16170  6.18391e-05  1 / 16,171
39    71756  795  8439  16174  0.000133741  13 / 97,203

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.999866
2    1
3    1
4    1
5    1

ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.027703007734643754
RMSE: 0.16644220538866863
LogLoss: 0.11495130715751167
Mean Per-Class Error: 0.36296091786191786
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1     0    1     2      Error       Rate
----  -----  ---  ----  -----  ----------  --------------
12    2      0    8     17     0.692308    27 / 39
0     71621  16   12    106    0.00186747  134 / 71,755
0     148    70   421   164    0.912827    733 / 803
0     1      84   7269  1081   0.138234    1,166 / 8,435
1     5      11   1108  15046  0.069569    1,125 / 16,171
13    71777  181  8818  16414  0.0327665   3,185 / 97,203

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.967234
2    0.993303
3    0.998992
4    0.999743
5    1

Cross-Validation Metrics Summary: 
                         mean       sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  ---------  -----------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.967223   0.000633369  0.966565      0.967903      0.966926      0.966821      0.967901
aic                      nan        0            nan           nan           nan           nan           nan
auc                      nan        0            nan           nan           nan           nan           nan
err                      0.0327768  0.000633369  0.0334345     0.0320971     0.0330744     0.033179      0.0320988
err_count                637.2      12.3167      650           624           643           645           624
loglikelihood            nan        0            nan           nan           nan           nan           nan
logloss                  0.121968   0.00416833   0.125997      0.119335      0.12555       0.122728      0.11623
max_per_class_error      0.934886   0.0503761    0.886364   

In [45]:
aml2.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GBM_grid_1_AutoML_2_20240625_211210_model_5,0.362961,0.114951,0.166442,0.027703
GBM_3_AutoML_2_20240625_211210,0.363859,0.0967993,0.162413,0.0263778
GBM_5_AutoML_2_20240625_211210,0.368795,0.0948453,0.163708,0.0268002
GBM_grid_1_AutoML_2_20240625_211210_model_2,0.36906,0.0957857,0.166927,0.0278646
GBM_4_AutoML_2_20240625_211210,0.369884,0.103738,0.164211,0.0269654
GBM_2_AutoML_2_20240625_211210,0.378629,0.0950525,0.163207,0.0266366
DRF_1_AutoML_2_20240625_211210,0.379909,0.130675,0.162974,0.0265607
GBM_1_AutoML_2_20240625_211210,0.381801,0.100454,0.166523,0.0277297
GBM_grid_1_AutoML_2_20240625_211210_model_3,0.39053,0.0954869,0.164254,0.0269795
GBM_grid_1_AutoML_2_20240625_211210_model_4,0.393178,0.115586,0.17358,0.0301302


In [46]:
#We need to get the predictions from the model: accuracy, precision, recall, f1-score
predictions = aml2.predict(test)
predictions = predictions.as_data_frame()
predictions = predictions['predict'].values
test = test.as_data_frame()
test = test['CATEGORIA_FACTURA'].values
#show the results
print("Accuracy: ", accuracy_score(test, predictions))
print("Precision: ", precision_score(test, predictions, average='weighted'))
print("Recall: ", recall_score(test, predictions, average='weighted'))
print("F1-Score: ", f1_score(test, predictions, average='weighted'))
print(classification_report(test, predictions))
print(confusion_matrix(test, predictions))

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Accuracy:  0.9683552117196823
Precision:  0.9650774126144608
Recall:  0.9683552117196823
F1-Score:  0.96604644773382
              precision    recall  f1-score   support

          -2       1.00      0.30      0.46        10
          -1       1.00      1.00      1.00     18012
           0       0.35      0.10      0.15       194
           1       0.85      0.86      0.85      2063
           2       0.92      0.94      0.93      4022

    accuracy                           0.97     24301
   macro avg       0.82      0.64      0.68     24301
weighted avg       0.97      0.97      0.97     24301

[[    3     0     0     1     6]
 [    0 17980     5     3    24]
 [    0    37    19    91    47]
 [    0     8    25  1765   265]
 [    0    24     5   228  3765]]


/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECHA_FACTURA' has levels not trained on: ["01/07/2019", "01/10/2020", "02/11/2021", "03/09/2018", "09/04/2020", "30/10/2023"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'LIQUIDO_FACTURA' has levels not trained on: ["0,1", "0,65", "1,48", "1,74", "10,57", "10,98", "100,59", "100,96", "1000", "1000,5", ...7359 not listed..., "996,06", "996,47", "996,77", "996,91", "997,15", "997,5", "998,73", "9989,89", "999,34", "999,69"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECHA_CONTABILIZACION' has levels not trained on: ["01/07/2019", "01/10/2020", "02/11/2021", "03/09/2018", "09/04/2020", "30/10/2023"]
  warnings.warn(w)
/Users/bonay

In [48]:
# Cargar el dataset
file_path = 'dataset/vima.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
label = 'CATEGORIA_FACTURA'

train = h2o.H2OFrame(train_data)
test = h2o.H2OFrame(test_data)
y = "CATEGORIA_FACTURA"
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()    
x = list(train.columns)
x.remove(y)

aml3 = H2OAutoML(max_models=20, seed=1, max_runtime_secs=1200, sort_metric='mean_per_class_error', distribution='multinomial')
aml3.train(x=x, y=y, training_frame=train)

/var/folders/xx/fj7dzt957rd1sg25bl4jv1b00000gn/T/ipykernel_7748/2276797217.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, quotechar='"', delimiter=";")


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
21:58:36.135: AutoML: XGBoost is not available; skipping it.
21:58:36.136: _train param, Dropping bad and constant columns: [COUNT_TIPOS_TRANSACCION, TIENE_DESCUENTO]

███████████████████
22:04:13.247: _train param, Dropping bad and constant columns: [TIENE_DESCUENTO]

████
22:05:43.814: _train param, Dropping bad and constant columns: [TIENE_DESCUENTO]

█████
22:07:10.868: _train param, Dropping bad and constant columns: [TIENE_DESCUENTO]

███
22:08:14.258: _train param, Dropping bad and constant columns: [TIENE_DESCUENTO]

██
22:09:04.503: _train param, Dropping bad and constant columns: [TIENE_DESCUENTO]

████
22:10:05.383: _train param, Dropping bad and constant columns: [TIENE_DESCUENTO]

█████
22:11:52.372: _train param, Dropping bad and constant columns: [TIENE_DESCUENTO]



Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_3_20240625_215836


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    66                 330                         1.20335e+07            10           10           10            34            949           510.142

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.007753104010369141
RMSE: 0.08805171213763614
LogLoss: 0.048788030677668015
Mean Per-Class Error: 0.003326283911009626
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1     0      1      2      Error        Rate
----  -----  -----  -----  -----  -----------  -------------
126   0      0      0      0      0            0 / 126
0     12296  0      0      1      8.13206e-05  1 / 12,297
1     0      46919  300    1      0.00639546   302 / 47,221
0     1      12     66029  481    0.007426     494 / 66,523
0     0      0      151    55188  0.00272864   151 / 55,339
127   12297  46931  66480  55671  0.00522297   948 / 181,506

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.994777
2    1
3    1
4    1
5    1

ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.06700874029067641
RMSE: 0.25886046490469805
LogLoss: 0.24587167304662483
Mean Per-Class Error: 0.1800149418549513
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1     0      1      2      Error       Rate
----  -----  -----  -----  -----  ----------  ----------------
47    0      0      69     10     0.626984    79 / 126
1     12275  0      14     7      0.00178905  22 / 12,297
1     0      45568  1529   123    0.0350056   1,653 / 47,221
12    65     363    60143  5940   0.0959067   6,380 / 66,523
2     0      27     7740   47570  0.140389    7,769 / 55,339
63    12340  45958  69495  53650  0.0876169   15,903 / 181,506

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.912383
2    0.991278
3    0.999355
4    0.999785
5    1

Cross-Validation Metrics Summary: 
                         mean       sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  ---------  ----------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.912576   0.00116188  0.912043      0.913859      0.912316      0.911049      0.913611
aic                      nan        0           nan           nan           nan           nan           nan
auc                      nan        0           nan           nan           nan           nan           nan
err                      0.0874241  0.00116188  0.0879566     0.0861409     0.0876835     0.0889507     0.0863888
err_count                3173.6     42.1877     3193          3127          3183          3229          3136
loglikelihood            nan        0           nan           nan           nan           nan           nan
logloss                  0.248505   0.00666156  0.245525      0.247824      0.250406      0.258412      0.240357
max_per_cl

In [49]:
aml3.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GBM_4_AutoML_3_20240625_215836,0.180015,0.245872,0.25886,0.0670087
GBM_2_AutoML_3_20240625_215836,0.180525,0.254878,0.258353,0.066746
GBM_5_AutoML_3_20240625_215836,0.180606,0.255563,0.259307,0.0672404
GBM_3_AutoML_3_20240625_215836,0.18094,0.255685,0.26223,0.0687647
GBM_1_AutoML_3_20240625_215836,0.185933,0.256489,0.265681,0.0705865
DRF_1_AutoML_3_20240625_215836,0.195252,0.326973,0.288686,0.0833396
XRT_1_AutoML_3_20240625_215836,0.329162,0.46118,0.379698,0.144171
GLM_1_AutoML_3_20240625_215836,0.45085,0.654734,0.485366,0.23558


In [50]:
#We need to get the predictions from the model: accuracy, precision, recall, f1-score
predictions = aml3.predict(test)
predictions = predictions.as_data_frame()
predictions = predictions['predict'].values
test = test.as_data_frame()
test = test['CATEGORIA_FACTURA'].values
#show the results
print("Accuracy: ", accuracy_score(test, predictions))
print("Precision: ", precision_score(test, predictions, average='weighted'))
print("Recall: ", recall_score(test, predictions, average='weighted'))
print("F1-Score: ", f1_score(test, predictions, average='weighted'))
print(classification_report(test, predictions))
print(confusion_matrix(test, predictions))

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Accuracy:  0.9237261988716502
Precision:  0.9246879498249929
Recall:  0.9237261988716502
F1-Score:  0.9239390724270719
              precision    recall  f1-score   support

          -2       0.80      0.40      0.53        30
          -1       0.99      1.00      0.99      3017
           0       0.99      0.96      0.98     11936
           1       0.88      0.92      0.90     16589
           2       0.90      0.88      0.89     13804

    accuracy                           0.92     45376
   macro avg       0.91      0.83      0.86     45376
weighted avg       0.92      0.92      0.92     45376

[[   12     0     0    17     1]
 [    1  3006     0     5     5]
 [    1     0 11494   410    31]
 [    1    22    87 15215  1264]
 [    0     0     2  1614 12188]]


/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'NUMERO_SERIE' has levels not trained on: ["FV5", "FVI"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECHA_FACTURA' has levels not trained on: ["02/02/2024", "07/01/2024"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'LIQUIDO_FACTURA' has levels not trained on: ["0,24", "0,55", "0,57", "0,81", "0,84", "1,05", "1,4", "100,3", "100,34", "100,42", ...14372 not listed..., "99772,07", "9980,19", "99857,74", "9986,05", "9987", "9988,4", "999,48", "9993,98", "9999,16", "9999,68"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECH

In [51]:
# Cargar el dataset
file_path = 'dataset/decepal.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
label = 'CATEGORIA_FACTURA'

train = h2o.H2OFrame(train_data)
test = h2o.H2OFrame(test_data)
y = "CATEGORIA_FACTURA"
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()    
x = list(train.columns)
x.remove(y)

aml4 = H2OAutoML(max_models=20, seed=1, max_runtime_secs=1200, sort_metric='mean_per_class_error', distribution='multinomial')
aml4.train(x=x, y=y, training_frame=train)

/var/folders/xx/fj7dzt957rd1sg25bl4jv1b00000gn/T/ipykernel_7748/3495445177.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, quotechar='"', delimiter=";")


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
22:18:42.209: AutoML: XGBoost is not available; skipping it.
22:18:42.211: _train param, Dropping bad and constant columns: [DIVISA, TIENE_DESCUENTO]

████████████████████
22:24:47.807: _train param, Dropping bad and constant columns: [DIVISA, TIENE_DESCUENTO]

██████████
22:27:57.749: _train param, Dropping bad and constant columns: [DIVISA, TIENE_DESCUENTO]

█████
22:29:43.467: _train param, Dropping bad and constant columns: [DIVISA, TIENE_DESCUENTO]

██████
22:31:32.446: _train param, Dropping bad and constant columns: [DIVISA, TIENE_DESCUENTO]

██████
22:33:37.319: _train param, Dropping bad and constant columns: [DIVISA, TIENE_DESCUENTO]

███████
22:36:02.784: _train param, Dropping bad and constant columns: [DIVISA, TIENE_DESCUENTO]

████████
22:38:24.226: _train param, Dro

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_4_20240625_221842


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    161                805                         1.62998e+07            0            15           10.7404       1             1295          481.431

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.008326875896090652
RMSE: 0.09125171722269479
LogLoss: 0.04433089120356707
Mean Per-Class Error: 0.010789968822448611
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1     0      1       2      Error        Rate
----  -----  -----  ------  -----  -----------  ---------------
47    0      0      0       0      0            0 / 47
0     19004  1      613     1      0.0313472    615 / 19,619
0     3      14968  1       324    0.0214435    328 / 15,296
0     80     0      128264  3      0.000646684  83 / 128,347
0     0      37     1       74111  0.000512482  38 / 74,149
47    19087  15006  128879  74439  0.00448079   1,064 / 237,458

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.995519
2    0.999987
3    1
4    1
5    1

ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.04787223452607481
RMSE: 0.21879724524334124
LogLoss: 0.17101329084622025
Mean Per-Class Error: 0.1363469301853393
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1     0      1       2      Error      Rate
----  -----  -----  ------  -----  ---------  ----------------
44    0      0      2       1      0.0638298  3 / 47
0     15064  67     4374    114    0.232173   4,555 / 19,619
0     230    10193  113     4760   0.333617   5,103 / 15,296
0     1414   40     126529  364    0.0141647  1,818 / 128,347
0     178    2308   328     71335  0.0379506  2,814 / 74,149
44    16886  12608  131346  76574  0.0601917  14,293 / 237,458

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.939808
2    0.99287
3    0.997557
4    0.999987
5    1

Cross-Validation Metrics Summary: 
                         mean       sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  ---------  ----------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.939813   0.00269474  0.94378       0.9394        0.940032      0.936198      0.939652
aic                      nan        0           nan           nan           nan           nan           nan
auc                      nan        0           nan           nan           nan           nan           nan
err                      0.0601875  0.00269474  0.05622       0.0605997     0.059968      0.0638016     0.0603483
err_count                2858.4     127.956     2670          2878          2848          3030          2866
loglikelihood            nan        0           nan           nan           nan           nan           nan
logloss                  0.171042   0.00566     0.163274      0.168771      0.171008      0.178519      

In [52]:
aml4.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GBM_1_AutoML_4_20240625_221842,0.136347,0.171013,0.218797,0.0478722
GBM_4_AutoML_4_20240625_221842,0.143832,0.177966,0.222963,0.0497125
GBM_3_AutoML_4_20240625_221842,0.15853,0.18522,0.229896,0.0528521
GBM_2_AutoML_4_20240625_221842,0.165777,0.19833,0.238667,0.0569617
DRF_1_AutoML_4_20240625_221842,0.202961,0.2509,0.264503,0.0699618
XRT_1_AutoML_4_20240625_221842,0.497282,0.485377,0.398867,0.159095
GLM_1_AutoML_4_20240625_221842,0.693457,0.832895,0.542955,0.2948


In [53]:
#We need to get the predictions from the model: accuracy, precision, recall, f1-score
predictions = aml4.predict(test)
predictions = predictions.as_data_frame()
predictions = predictions['predict'].values
test = test.as_data_frame()
test = test['CATEGORIA_FACTURA'].values
#show the results
print("Accuracy: ", accuracy_score(test, predictions))
print("Precision: ", precision_score(test, predictions, average='weighted'))
print("Recall: ", recall_score(test, predictions, average='weighted'))
print("F1-Score: ", f1_score(test, predictions, average='weighted'))
print(classification_report(test, predictions))
print(confusion_matrix(test, predictions))

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECHA_FACTURA' has levels not trained on: ["12/03/2022", "25/12/2019"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'LIQUIDO_FACTURA' has levels not trained on: ["0,14", "1,37", "1001,27", "1001,82", "1002,59", "1002,63", "1002,77", "1003,78", "1004,14", "1004,68", ...6265 not listed..., "996,72", "996,93", "997,12", "997,46", "997,81", "9978,15", "998,3", "998,88", "999,46", "999,6"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECHA_CONTABILIZACION' has levels not trained on: ["12/03/2022", "25/12/2019"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Te

Accuracy:  0.9405195067717809
Precision:  0.9383796288809501
Recall:  0.9405195067717809
F1-Score:  0.938155504692226
              precision    recall  f1-score   support

          -2       1.00      0.88      0.93         8
          -1       0.90      0.76      0.83      4922
           0       0.83      0.66      0.74      3822
           1       0.96      0.99      0.97     32082
           2       0.93      0.97      0.95     18530

    accuracy                           0.94     59364
   macro avg       0.92      0.85      0.88     59364
weighted avg       0.94      0.94      0.94     59364

[[    7     0     0     1     0]
 [    0  3765     8  1124    25]
 [    0    44  2512    24  1242]
 [    0   346    11 31627    98]
 [    0    49   481    78 17922]]


In [58]:
# Cargar el dataset
file_path = 'dataset/suco.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
label = 'CATEGORIA_FACTURA'

train = h2o.H2OFrame(train_data)
test = h2o.H2OFrame(test_data)
y = "CATEGORIA_FACTURA"
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()    
x = list(train.columns)
x.remove(y)

aml5 = H2OAutoML(max_models=20, seed=1, max_runtime_secs=300, sort_metric='mean_per_class_error', distribution='multinomial')
aml5.train(x=x, y=y, training_frame=train)

/var/folders/xx/fj7dzt957rd1sg25bl4jv1b00000gn/T/ipykernel_7748/2618182701.py:3: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, quotechar='"', delimiter=";")


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
08:06:52.275: AutoML: XGBoost is not available; skipping it.
08:06:52.329: _train param, Dropping bad and constant columns: [COUNT_TIPOS_TRANSACCION]

██████████████████████████████████████████████████
08:10:50.458: _train param, Dropping bad and constant columns: [COUNT_TIPOS_TRANSACCION]

█████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_1_20240626_80652


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    360                1440                        3.86171e+06            7            7            7             12            128           105.638

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.015151261808977964
RMSE: 0.12309046189278015
LogLoss: 0.06358166221102228
Mean Per-Class Error: 0.016618862175117398
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1     0      1       2      Error      Rate
----  -----  -----  ------  -----  ---------  ---------------
0     0      0      0       0      nan        0 / 0
0     58014  0      0       0      0          0 / 58,014
0     0      27372  690     37     0.0258728  727 / 28,099
0     0      1717   194647  443    0.0109752  2,160 / 196,807
0     0      217    1230    29842  0.0462463  1,447 / 31,289
0     58014  29306  196567  30322  0.0137934  4,334 / 314,209

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.986207
2    0.999319
3    0.999994
4    1
5    1

ModelMetricsMultinomial: gbm
** Reported on validation data. **

MSE: 0.05153176288148799
RMSE: 0.22700608556047125
LogLoss: 0.18358121752206283
Mean Per-Class Error: 0.09003031802604364
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
-2    -1    0     1      2     Error       Rate
----  ----  ----  -----  ----  ----------  --------------
0     0     0     0      0     nan         0 / 0
0     8222  0     1      0     0.00012161  1 / 8,223
0     0     3547  430    34    0.115682    464 / 4,011
0     0     598   27073  423   0.0363423   1,021 / 28,094
0     0     106   1224   3133  0.298006    1,330 / 4,463
0     8222  4251  28728  3590  0.0628698   2,816 / 44,791

Top-5 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.93713
2    0.992231
3    0.998571
4    1
5    1

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse        training_logloss     training_classification_error    training_auc    training_pr_auc    validation_rmse      validation_logloss    validation_classification_error    validation_auc    validation_pr_auc
---  -------------------  ----------  -----------------  -------------------  -------------------  -------------------------------  --------------  -----------------  -------------------  --------------------  ---------------------------------  ----------------  -------------------
     2024-06-26 08:08:26  0.001 sec   0.0                0.799999999999844    1.6094379124335148   0.5555824308024276               nan             nan                0.8000000000000056   1.6094379124341998    0.5550445401978076                 nan               nan
     2024-06-26 08:08:27  0.375 sec   5.0                0.5542533086185437   0.8287467293226504   0.11945233904821313              nan             nan                0.5554296530255176   0.8322504628104452    0.12067156348373557           

In [59]:
aml5.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GBM_2_AutoML_1_20240626_80652,0.0900303,0.183581,0.227006,0.0515318
GBM_3_AutoML_1_20240626_80652,0.0905416,0.188477,0.227444,0.051731
GBM_5_AutoML_1_20240626_80652,0.0915209,0.184423,0.228492,0.0522086
GBM_4_AutoML_1_20240626_80652,0.0940894,0.198483,0.229904,0.052856
GBM_1_AutoML_1_20240626_80652,0.0959975,0.2145,0.232962,0.0542715
DRF_1_AutoML_1_20240626_80652,0.110218,0.192438,0.238228,0.0567526
XRT_1_AutoML_1_20240626_80652,0.157185,0.241106,0.270827,0.0733471
DeepLearning_1_AutoML_1_20240626_80652,0.418955,0.742203,0.49801,0.248014
GLM_1_AutoML_1_20240626_80652,0.6,1.04963,0.603863,0.364651


In [60]:
#We need to get the predictions from the model: accuracy, precision, recall, f1-score
predictions = aml5.predict(test)
predictions = predictions.as_data_frame()
predictions = predictions['predict'].values
test = test.as_data_frame()
test = test['CATEGORIA_FACTURA'].values
#show the results
print("Accuracy: ", accuracy_score(test, predictions))
print("Precision: ", precision_score(test, predictions, average='weighted'))
print("Recall: ", recall_score(test, predictions, average='weighted'))
print("F1-Score: ", f1_score(test, predictions, average='weighted'))
print(classification_report(test, predictions))
print(confusion_matrix(test, predictions))

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECHA_FACTURA' has levels not trained on: ["01/05/2023"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'LIQUIDO_FACTURA' has levels not trained on: ["0,24", "0,31", "0,39", "0,61", "0,99", "1,07", "1,1", "10,44", "100,38", "100,53", ...47649 not listed..., "9995,05", "9995767", "9996,5", "99961,21", "9997,14", "9997,64", "9998,52", "9999,06", "9999279,35", "99994,95"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'FECHA_CONTABILIZACION' has levels not trained on: ["01/05/2023"]
  warnings.warn(w)
/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'MA

Accuracy:  0.9366498493573173
Precision:  0.9358805531011544
Recall:  0.9366498493573173
F1-Score:  0.9354662681466709
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00     20624
           0       0.83      0.87      0.85      9784
           1       0.94      0.96      0.95     70661
           2       0.86      0.72      0.78     11117

    accuracy                           0.94    112186
   macro avg       0.91      0.89      0.90    112186
weighted avg       0.94      0.94      0.94    112186

[[20621     0     3     0]
 [    0  8543  1153    88]
 [    2  1533 67959  1167]
 [    0   269  2892  7956]]


In [61]:
lb = h2o.automl.get_leaderboard(aml5, extra_columns = "ALL")
lb

model_id,mean_per_class_error,logloss,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
GBM_2_AutoML_1_20240626_80652,0.0900303,0.183581,0.227006,0.0515318,27859,0.045628,GBM
GBM_3_AutoML_1_20240626_80652,0.0905416,0.188477,0.227444,0.051731,22108,0.028947,GBM
GBM_5_AutoML_1_20240626_80652,0.0915209,0.184423,0.228492,0.0522086,38452,0.059552,GBM
GBM_4_AutoML_1_20240626_80652,0.0940894,0.198483,0.229904,0.052856,17312,0.017982,GBM
GBM_1_AutoML_1_20240626_80652,0.0959975,0.2145,0.232962,0.0542715,20662,0.0147,GBM
DRF_1_AutoML_1_20240626_80652,0.110218,0.192438,0.238228,0.0567526,18376,0.008079,DRF
XRT_1_AutoML_1_20240626_80652,0.157185,0.241106,0.270827,0.0733471,37702,0.008611,DRF
DeepLearning_1_AutoML_1_20240626_80652,0.418955,0.742203,0.49801,0.248014,62323,0.19896,DeepLearning
GLM_1_AutoML_1_20240626_80652,0.6,1.04963,0.603863,0.364651,54782,8.9e-05,GLM
